# Hashing

Hashing and equivalence are tightly coupled.

Immutable objects such as strings can sometimes bind to the same object.

In [176]:
a = "Hello"
b = "Hello"

print(f"a is at {id(a)} ")
print(f"b is at {id(b)} ")
print("a is b:", a is b)
print(f"a == b:", a == b)

a is at 1706296023792 
b is at 1706296023792 
a is b: True
a == b: True


In built mutuable objects such as lists with same value are different objects.

In [177]:
a = [1,2,3]
b = [1,2,3]

print(f"a is at {id(a)} ")
print(f"b is at {id(b)} ")
print("a is b:", a is b)
print("a == b:", a == b)

a is at 1706296246208 
b is at 1706296456768 
a is b: False
a == b: True


Mutable user objects are not the same object and they are, by default, not equivalent.

In [178]:
class MyClass:
    def __init__(self, name):
        self.name = name

a = MyClass("My string")  # Two instances look equivalent
b = MyClass("My string")

print(f"a is at {id(a)} ")
print(f"b is at {id(b)} ")
print(f"a is b:", a is b)
print(f"a == b:", a == b)

a is at 1706307185792 
b is at 1706307178784 
a is b: False
a == b: False


Mutable user objects that implement `__eq__` are not the same but can be equivalent......

In [179]:
class MyEquivClass:
    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        return self.name == other.name

a = MyEquivClass("My string")
b = MyEquivClass("My string")

print("a is b:", a is b)
print("a == b:", a == b)

a is b: False
a == b: True


These different types are also different in terms of hashability. If an object is hashable it means it can be used a key in a dictionary.

In [180]:
print(hash(-109))
print(hash('My string'))
print('My string'.__hash__())

-109
-289870689659658676
-289870689659658676


Immutable in built objects can be used as a dictionary key

In [181]:
my_dict = {'my string!': 45, 'my other string': True, 1: 0.5}
print(my_dict)

{'my string!': 45, 'my other string': True, 1: 0.5}


...but mutable in built objects are not hashable

In [182]:
try:
    hash([1,2,3])
except TypeError:
    print("In-built mutable objects are not hashable")

try:
    my_dict = {[1,2,3]: True}
except:
    print("In-built mutable objects can not be keys")


In-built mutable objects are not hashable
In-built mutable objects can not be keys


What about user defined mutable classes?

In [183]:
print(hash(MyClass))
print(hash(MyEquivClass))
m_dict = {MyClass: 'Hello class'}
print(m_dict)
m_dict = {MyEquivClass: 'Hello class'}
print(m_dict)

106642310035
106642310095
{<class '__main__.MyClass'>: 'Hello class'}
{<class '__main__.MyEquivClass'>: 'Hello class'}


Looks like they are hashable and can be used in a dictionary. Default user defined mutable class instances are also hashable. This is because the hash is dependent on their `id()`. However, if you implement `__eq__` you must also implement `__hash__`.

In [184]:
print(hash(MyClass('my string')))
try:
    hash(MyEquivClass('my string'))
except TypeError:
    print("User objects that only implement __eq__ are not hashable")

106644199082
User objects that only implement __eq__ are not hashable


If `__eq__` and `__hash__` are implemented mutable class instances are also hashable. As such they can be used in dictionaries as keys. Equivalent objects can be used to update the dictionary.

In [185]:
class MyEquivAndHashClass:
    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        return self.name == other.name

    def __hash__(self):
        return hash(self.name)

print(hash(MyEquivAndHashClass('Bob')))
a = MyEquivAndHashClass('Bob')
b = MyEquivAndHashClass('Bob')
print("a is at", id(a))
print("b is at", id(b))
print("a == b", a == b)
my_dict = {a: True}           # key is object a
print(my_dict)
my_dict[a] = False            # a is same as key in dictionary
print(my_dict)
my_dict[b] = True             # same hash and equivalent so updating value for key based on object a
print(my_dict)

6946763540305283504
a is at 1706307175376
b is at 1706307179408
a == b True
{<__main__.MyEquivAndHashClass object at 0x0000018D47D543D0>: True}
{<__main__.MyEquivAndHashClass object at 0x0000018D47D543D0>: False}
{<__main__.MyEquivAndHashClass object at 0x0000018D47D543D0>: True}


So what happens to the dictionary if the key changes?

In [186]:
a.name = 'Tim'
print("a == b", a == b)
print(my_dict)
try:
    condition = my_dict[a]
except KeyError:
    print("However, key is now different (different hash) so will not be found in dictionary")

a == b False
{<__main__.MyEquivAndHashClass object at 0x0000018D47D543D0>: True}
However, key is now different (different hash) so will not be found in dictionary


If you try to update dictionary a new key-pair will be created. This will be slightly weird as we now have two keys for same object id but ok as we are saying Tim and Bob are not the same person so use different key.

In [187]:
my_dict[a] = False         # hash and equivalence do not match any existing key so add new entry
print(my_dict)

{<__main__.MyEquivAndHashClass object at 0x0000018D47D543D0>: True, <__main__.MyEquivAndHashClass object at 0x0000018D47D543D0>: False}


 If we were to set `a.name` back to one then we could change the first entry in the dictionary using `a` as the key. However, instead we update the dictionary using the dictionary using `b` as the key. This creates a further new entry.

In [188]:
my_dict[b] = 'Changed from bool to string'  # no match with either existing keys as both 'Tim' so new entry
print(my_dict)

{<__main__.MyEquivAndHashClass object at 0x0000018D47D543D0>: True, <__main__.MyEquivAndHashClass object at 0x0000018D47D543D0>: False, <__main__.MyEquivAndHashClass object at 0x0000018D47D55390>: 'Changed from bool to string'}


So what happens when two objects have different hash computation to equivalence?

In [189]:
# but you need hash and equivalence to use same information
# else weird stuff will happen
class MyBadEquivAndHashClass:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __eq__(self, other):
        return self.name == other.name

    def __hash__(self):
        return hash(self.name+str(self.age))

a = MyBadEquivAndHashClass('Bob', 15)
b = MyBadEquivAndHashClass('Bob', 16)
print("a == b:", a == b)
my_dict = {a: True}
print(my_dict)
my_dict[a] = False
print(my_dict)
my_dict[b] = True
print(my_dict)

a == b: True
{<__main__.MyBadEquivAndHashClass object at 0x0000018D47D559C0>: True}
{<__main__.MyBadEquivAndHashClass object at 0x0000018D47D559C0>: False}
{<__main__.MyBadEquivAndHashClass object at 0x0000018D47D559C0>: False, <__main__.MyBadEquivAndHashClass object at 0x0000018D47D56A40>: True}


So this weird as `a` and `b` are supposedly equivalent but now 2 keyed items in the dictionary.

If you want to use a mutable user object as a dictionary key then `__hash__` and `__eq__` should match modifiable entities!

In [190]:
class MyGoodEquivAndHashClass:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __eq__(self, other):
        return self.name == other.name and self.age == other.age

    def __hash__(self):
        return hash(self.name+str(self.age))

a = MyGoodEquivAndHashClass('Bob', 15)
b = MyGoodEquivAndHashClass('Bob', 15)
print("a == b:", a == b)
my_dict = {a: 0}
my_dict[a] = 1
my_dict[b] = 2
print(my_dict)

a == b: True
{<__main__.MyGoodEquivAndHashClass object at 0x0000018D47D57A90>: 2}


So this looks a bit better as we have only one entry in the dictionary and we've used both instances as indexes into the dictionary. What happens if we set age of one of the instances to 16?

In [191]:
a.age = 16
my_dict[a] = 3
print(my_dict)
print(my_dict[a])
a.age = 15
print(my_dict[a])

{<__main__.MyGoodEquivAndHashClass object at 0x0000018D47D57A90>: 2, <__main__.MyGoodEquivAndHashClass object at 0x0000018D47D57A90>: 3}
3
2


So in this example we are saying 15 year old Bob is treated differently in this dictionary to 16 year Bob.

Dictionaries work by using hashing/has tables. The key is converted to a hash code.

A quick reminder on how hashing works. As fundamental to Python as nearly everything under hood is a dict. If we have 100000 telephone number records is a linear array and we want to find the record for a particular number we can linearly search. if first in array great, if last when search time will be long a solution is to apply a function to the number/string whatever, a hashing function the resulting value, an integer, indexes into a hash table a hash table has a number or rows, or buckets the bucket index is usually computed as `hash_value % number buckets` each bucket may have a small number of phone records in it so when I want to do a look-up I hash the number (key) which gives me a bucket I then go along row (chain) to find record, or pointer to record, associated with my hash value if there is a collision - two keys generate same hash value then need to compare actual numbers to find right record so choice of hash function is important to spread records evenly over all the buckets and also to avoid collisions.
